 - How to scale current manual processes and resources to screen 500,000 projects so that they can be posted as quickly and as efficiently as possible
 - How to increase the consistency of project vetting across different volunteers to improve the experience for teachers
 - How to focus volunteer time on the applications that need the most assistance

#### The goal of the competition is to predict whether or not a DonorsChoose.org project proposal submitted by a teacher will be approved, using the text of project descriptions as well as additional metadata about the project, teacher, and school. DonorsChoose.org can then use this information to identify projects most likely to need further review before approval.

In [1]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')
resource = pd.read_csv('data/resources.csv')

C:\Users\user\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
train.columns

Index(['id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_essay_1', 'project_essay_2',
       'project_essay_3', 'project_essay_4', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved'],
      dtype='object')

#### For all projects with project_submitted_datetime of 2016-05-17 and later, the values of project_essay_3 and project_essay_4 will be NaN.

In [9]:
train.head()

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved
0,p036502,484aaf11257089a66cfedc9461c6bd0a,Ms.,NV,2016-11-18 14:45:59,Grades PreK-2,Literacy & Language,Literacy,Super Sight Word Centers,Most of my kindergarten students come from low...,I currently have a differentiated sight word c...,NaN,NaN,My students need 6 Ipod Nano's to create and d...,26,1
1,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0
2,p233823,a9b876a9252e08a55e3d894150f75ba3,Ms.,UT,2017-01-01 22:57:44,Grades 3-5,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Lets 3Doodle to Learn,Hello;\r\nMy name is Mrs. Brotherton. I teach ...,We are looking to add some 3Doodler to our cla...,NaN,NaN,My students need the 3doodler. We are an SEM s...,5,1
3,p185307,525fdbb6ec7f538a48beebaa0a51b24f,Mr.,NC,2016-08-12 15:42:11,Grades 3-5,Health & Sports,Health & Wellness,"\""Kid Inspired\"" Equipment to Increase Activit...",My students are the greatest students but are ...,"The student's project which is totally \""kid-i...",NaN,NaN,My students need balls and other activity equi...,16,0
4,p013780,a63b5547a7239eae4c1872670848e61a,Mr.,CA,2016-08-06 09:09:11,Grades 6-8,Health & Sports,Health & Wellness,We need clean water for our culinary arts class!,My students are athletes and students who are ...,For some reason in our kitchen the water comes...,NaN,NaN,My students need a water filtration system for...,42,1


In [18]:
train.school_state.value_counts()

CA    25695
TX    12304
NY    12157
FL    10359
NC     8463
IL     7332
GA     6636
SC     6463
MI     5324
PA     5093
IN     4314
MO     4247
OH     4139
MA     4054
LA     3946
WA     3903
OK     3829
NJ     3671
AZ     3614
VA     3446
WI     2983
AL     2955
TN     2836
UT     2814
CT     2766
MD     2480
NV     2297
MS     2222
KY     2172
OR     2119
MN     2055
CO     1887
AR     1757
ID     1113
IA     1079
KS     1060
NM      916
DC      902
HI      869
WV      839
ME      827
DE      589
NH      587
AK      557
SD      494
NE      493
RI      475
MT      384
ND      248
WY      177
VT      139
Name: school_state, dtype: int64

In [20]:
train.project_subject_subcategories.value_counts()

Literacy                                        15775
Literacy, Mathematics                           13863
Literature & Writing, Mathematics                9976
Literacy, Literature & Writing                   9282
Mathematics                                      9041
Literature & Writing                             7474
Special Needs                                    7065
Health & Wellness                                5924
Applied Sciences, Mathematics                    5651
Literacy, Special Needs                          4091
Applied Sciences                                 4037
Gym & Fitness, Health & Wellness                 3864
Visual Arts                                      3671
ESL, Literacy                                    3624
Music                                            2427
Literature & Writing, Special Needs              2244
Warmth, Care & Hunger                            2191
Health & Wellness, Special Needs                 2045
Gym & Fitness               

In [11]:
train.project_subject_categories.value_counts()

Literacy & Language                           39257
Math & Science                                28555
Literacy & Language, Math & Science           24499
Health & Sports                               16951
Music & The Arts                               8527
Special Needs                                  7065
Literacy & Language, Special Needs             6685
Applied Learning                               6310
Math & Science, Literacy & Language            3843
Applied Learning, Literacy & Language          3725
History & Civics                               3065
Math & Science, Special Needs                  3010
Literacy & Language, Music & The Arts          2878
Math & Science, Music & The Arts               2761
Applied Learning, Special Needs                2481
Health & Sports, Special Needs                 2368
History & Civics, Literacy & Language          2288
Warmth, Care & Hunger                          2191
Math & Science, Applied Learning               2071
Applied Lear

In [10]:
train.project_grade_category.value_counts()

Grades PreK-2    73890
Grades 3-5       61682
Grades 6-8       28197
Grades 9-12      18311
Name: project_grade_category, dtype: int64

In [8]:
train.project_resource_summary.value_counts()

My students need electronic tablets to do all the things that will make them successful in this 21st century learning environment.                                                                                          84
My students need Chromebooks to do all the things that will make them successful in this 21st century learning environment.                                                                                                 19
My students need chromebooks to do all the things that will make them successful in this 21st century learning environment.                                                                                                 11
My students need a Dell Chromebook 3120 and a Google EDU Management Console License.                                                                                                                                         9
My students need a Dell Chromebook and Google EDU Management Console License.                               

In [30]:
resource.head()

,id,description,quantity,price
0,p233245,LC652 - Lakeshore Double-Space Mobile Drying Rack,1,149.00
1,p069063,Bouncy Bands for Desks (Blue support pipes),3,14.95
2,p069063,Cory Stories: A Kid's Book About Living With Adhd,1,8.45
3,p069063,"Dixon Ticonderoga Wood-Cased #2 HB Pencils, Bo...",2,13.59
4,p069063,EDUCATIONAL INSIGHTS FLUORESCENT LIGHT FILTERS...,3,24.95


In [37]:
train.shape, test.shape, sample_submission.shape, resource.shape

((182080, 16), (78035, 15), (78035, 2), (1541272, 4))

In [42]:
train.id.value_counts()

p151963    1
p163158    1
p098600    1
p193384    1
p054272    1
p146776    1
p026207    1
p256245    1
p019788    1
p052128    1
p172774    1
p100575    1
p233607    1
p250285    1
p154381    1
p041151    1
p228654    1
p229512    1
p179445    1
p166298    1
p032154    1
p141513    1
p170260    1
p104291    1
p138907    1
p024332    1
p056695    1
p169735    1
p107622    1
p242166    1
          ..
p106482    1
p185477    1
p147660    1
p223805    1
p166231    1
p050589    1
p057748    1
p017240    1
p191941    1
p253497    1
p107415    1
p196280    1
p241981    1
p007862    1
p105424    1
p169180    1
p156283    1
p173070    1
p250819    1
p152173    1
p010547    1
p014767    1
p048910    1
p219997    1
p182211    1
p250374    1
p009407    1
p221523    1
p248696    1
p020145    1
Name: id, Length: 182080, dtype: int64

In [56]:
train.loc[25499]

id                                                                                        p066966
teacher_id                                                       4e8fb3d732f4a275b5e24c2ed9d8ebd4
teacher_prefix                                                                                Ms.
school_state                                                                                   SC
project_submitted_datetime                                                    2017-04-03 14:35:31
project_grade_category                                                              Grades PreK-2
project_subject_categories                                                    Literacy & Language
project_subject_subcategories                                                            Literacy
project_title                                                Leveled Books for Struggling Readers
project_essay_1                                 My students are group of energetic, loving, sm...
project_essay_2     

In [55]:
train.loc[25499].project_resource_summary

'My students need in-class library books on guided reading levels A-C for independent reading time.'

In [48]:
resource[resource['id'] == 'p066966']

,id,description,quantity,price
1481843,p066966,1 Hunter,1,5.10
1481844,p066966,A Picnic,1,2.55
1481845,p066966,A Rainbow of My Own,1,5.10
1481846,p066966,About Dogs,1,7.26
1481847,p066966,Above and Below,1,3.64
1481848,p066966,All Through The Week With Cat And Dog,1,2.55
1481849,p066966,Animal Babies,1,4.56
1481850,p066966,Animals in Winter,1,4.37
1481851,p066966,Apples,1,5.80
1481852,p066966,Astronaut,1,5.80


p066966    100
p137603    100
p009778    100
p121858     99
p143469     99
p055385     99
p108080     99
p218339     98
p135784     97
p086644     97
p234175     96
p010076     96
p000004     95
p153169     95
p200152     95
p161172     95
p094292     95
p104836     95
p231866     95
p219650     95
p000773     95
p102427     94
p004377     94
p089108     94
p101917     94
p098840     94
p210324     94
p231257     93
p092663     93
p021245     93
          ... 
p194909      1
p242685      1
p226681      1
p130499      1
p237183      1
p154206      1
p186917      1
p122678      1
p000524      1
p173321      1
p061375      1
p231381      1
p243473      1
p067984      1
p020080      1
p135708      1
p119337      1
p079856      1
p219215      1
p194322      1
p037479      1
p114958      1
p089779      1
p152387      1
p096349      1
p155172      1
p218151      1
p025280      1
p242961      1
p214417      1
Name: id, Length: 260115, dtype: int64